In [ ]:
#rm -rf /kaggle/working/*

In [ ]:
!git clone --branch baselineCe https://github.com/Graph-Classification-Noisy-Label/hackaton.git

In [ ]:
!git clone --branch main https://github.com/giankev/GNN_classification_noisy_label.git

In [ ]:
%cd hackaton/
!gdown --folder https://drive.google.com/drive/folders/1Z-1JkPJ6q4C6jX4brvq1VRbJH5RPUCAk -O datasets
!ls -lh datasets

In [ ]:
!pip install torch_geometric

In [ ]:
# source/utils.py
import random
import torch
import numpy as np
import logging
from typing import Dict, Any

def set_seed(seed: int):
    random.seed(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

In [ ]:
import gzip
import json
import os
import pandas as pd
from torch_geometric.data import Data, DataLoader
import torch
from sklearn.model_selection import train_test_split

def load_dataset(file_path: str) -> pd.DataFrame:
    data = []
    db = []    
    for file in file_path.split(' '):
        x = os.path.basename(os.path.dirname(file))
        with gzip.open(file, 'rt', encoding='utf-8') as f:
            tmp = json.load(f)
            data = data + tmp
            db = db + [x]*len(tmp)
    data = pd.DataFrame(data)
    data = data.assign(db=db)
    return data

def create_dataset_from_dataframe(df, result=True):
    dataset = []
    for _, row in df.iterrows():
        edge_index = torch.tensor(row['edge_index'], dtype=torch.long)
        edge_attr = torch.tensor(row['edge_attr'], dtype=torch.float)  
        num_nodes = row['num_nodes']

        # Safe handling of y
        y_raw = row.get('y', None)
        if result and y_raw is not None and isinstance(y_raw, list) and len(y_raw) > 0 and isinstance(y_raw[0], list):
            y = torch.tensor([y_raw[0][0]], dtype=torch.long)
        else:
            y = torch.tensor([0], dtype=torch.long)

        # Create a Data object
        data = Data(
            x=torch.ones((num_nodes, 1)), 
            edge_index=edge_index,
            edge_attr=edge_attr,
            y=y
        )

        # Replace any NaNs with zeros
        data.x = torch.nan_to_num(data.x, nan=0.0)
        data.edge_attr = torch.nan_to_num(data.edge_attr, nan=0.0)

        dataset.append(data)
    return dataset

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SymmetricCrossEntropyLoss(nn.Module):
    def __init__(self, num_classes: int = 6, alpha: float = 1.0, beta: float = 1.0, eps: float = 1e-7):
        super().__init__()
        self.alpha = alpha
        self.beta = beta
        self.num_classes = num_classes
        self.eps = eps
        self.ce = nn.CrossEntropyLoss()

    def forward(self, logits: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:

        loss_ce = self.ce(logits, targets)

        prob = F.softmax(logits, dim=1)
        one_hot = F.one_hot(targets, self.num_classes).float().clamp(min=self.eps)
        loss_rce = (- prob * torch.log(one_hot)).sum(dim=1).mean()

        return self.alpha * loss_ce + self.beta * loss_rce

In [ ]:
class NoisyCrossEntropyLoss(torch.nn.Module):
    def __init__(self, p_noisy = 0.15):
        super().__init__()
        self.p = p_noisy
        self.ce = torch.nn.CrossEntropyLoss(reduction='none')

    def forward(self, logits, targets):
        losses = self.ce(logits, targets)
        weights = (1 - self.p) + self.p * (1 - torch.nn.functional.one_hot(targets, num_classes=logits.size(1)).float().sum(dim=1))
        return (losses * weights).mean()

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing, global_mean_pool


class EdgeEncoder(MessagePassing):
    def __init__(self, in_channels, edge_dim, hidden_dim):
        super(EdgeEncoder, self).__init__(aggr='add') 
        self.node_mlp = torch.nn.Linear(in_channels + hidden_dim, hidden_dim)
        self.edge_mlp = torch.nn.Sequential(
            torch.nn.Linear(edge_dim, hidden_dim),
            torch.nn.LeakyReLU(0.15),
            torch.nn.Linear(hidden_dim, hidden_dim)
        )

    def forward(self, x, edge_index, edge_attr):
        edge_emb = self.edge_mlp(edge_attr)  
        return self.propagate(edge_index, x=x, edge_attr=edge_emb)

    def message(self, x_i, x_j, edge_attr):
        z = torch.cat([x_i, edge_attr], dim=1)  
        return self.node_mlp(z)

    def update(self, aggr_out):
        return aggr_out
    
class EdgeVGAE(torch.nn.Module):
    def __init__(self, input_dim, edge_dim, hidden_dim, latent_dim, num_classes):
        super(EdgeVGAE, self).__init__()
        self.encoder = EdgeVGAEEncoder(input_dim, edge_dim, hidden_dim, latent_dim)
        self.classifier = torch.nn.Linear(latent_dim, num_classes) 
        
        self.edge_mlp = torch.nn.Sequential(
            torch.nn.Linear(latent_dim * 2, latent_dim),
            torch.nn.LeakyReLU(0.15),
            torch.nn.Linear(latent_dim, edge_dim)
        )

        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, torch.nn.Linear):
                torch.nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='leaky_relu', a=0.15)
                if m.bias is not None:
                    torch.nn.init.constant_(m.bias, 0)
                    
    def forward(self, x, edge_index, edge_attr, batch, eps=None):
        mu, logvar = self.encoder(x, edge_index, edge_attr)
        if eps==0.0:
            z = mu
        else:
            z = self.reparameterize(mu, logvar) 

        class_logits = self.classifier(global_mean_pool(z, batch))  
        return z, mu, logvar, class_logits

    def reparameterize(self, mu, logvar):
        logvar = torch.clamp(logvar, min=-10, max=10)  
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z, edge_index):
        adj_pred = torch.sigmoid(torch.mm(z, z.t()))

        row, col = edge_index
        edge_features = torch.cat([z[row], z[col]], dim=-1)  
        edge_attr_pred = self.edge_mlp(edge_features)
        edge_attr_pred = torch.sigmoid(edge_attr_pred)  
        
        return adj_pred, edge_attr_pred
    
    def recon_loss(self, z, edge_index, edge_attr):
        adj_pred, edge_attr_pred = self.decode(z, edge_index)

        # Build adjacency ground truth
        adj_true = torch.zeros_like(adj_pred, dtype=torch.float32)
        adj_true[edge_index[0], edge_index[1]] = 1.0  
        adj_loss = F.binary_cross_entropy(adj_pred, adj_true)

        edge_attr_pred_selected = edge_attr_pred  
        edge_loss = F.mse_loss(edge_attr_pred_selected, edge_attr)
        #return adj_loss
        return 0.1*adj_loss + edge_loss

    def kl_loss(self, mu, logvar):
        logvar = torch.clamp(logvar, min=-10, max=10)  
        return -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())

class EdgeVGAEEncoder(torch.nn.Module):
    def __init__(self, input_dim, edge_dim, hidden_dim, latent_dim):
        super(EdgeVGAEEncoder, self).__init__()
        self.conv1 = EdgeEncoder(input_dim, edge_dim, hidden_dim)
        self.conv2 = EdgeEncoder(hidden_dim, edge_dim, hidden_dim)
        self.drop = torch.nn.Dropout(0.05)

        # Mean and log variance layers
        self.mu_layer = torch.nn.Linear(hidden_dim, latent_dim)
        self.logvar_layer = torch.nn.Linear(hidden_dim, latent_dim)

    def forward(self, x, edge_index, edge_attr):
        x = self.drop(x)
        x = F.leaky_relu(self.conv1(x, edge_index, edge_attr), 0.15)
        x = self.drop(x)
        x = F.leaky_relu(self.conv2(x, edge_index, edge_attr), 0.15)
        # x = self.drop(x)
        return self.mu_layer(x), self.logvar_layer(x)  

In [ ]:
import sys
import types

config = types.ModuleType("config")

import dataclasses
import os
from typing import Optional

@dataclasses.dataclass
class ModelConfig:
    test_path:  Optional[str] = None
    train_path: Optional[str] = None
    pretrain_paths: Optional[str] = None
    batch_size: int = 32
    hidden_dim: int = 128
    latent_dim: int = 8
    num_classes: int = 6
    epochs: int = 50
    learning_rate: float = 0.00005
    num_cycles: int = 5
    warmup: int = 5
    early_stopping_patience: int = 20

    @property
    def folder_name(self) -> str:
        files = self.train_path if self.train_path is not None else self.test_path
        db = ''
        for file in files.split(' '):
            db += os.path.basename(os.path.dirname(file))
        return db

config.ModelConfig = ModelConfig

sys.modules["config"] = config

In [ ]:
import logging
import os
from datetime import datetime
from typing import List, Dict

import pandas as pd
import numpy as np
import torch

from torch_geometric.loader import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm
from sklearn.metrics import accuracy_score


def warm_up_lr(epoch, num_epoch_warm_up, init_lr, optimizer):
    for params in optimizer.param_groups:
        params['lr'] = (epoch+1)**3 * init_lr / num_epoch_warm_up**3

class ModelTrainer:
    def __init__(self, config: 'ModelConfig', device: str):
        self.config = config
        self.device = device
        self.models: List[str] = []
        self.pretrain_models: List[str] = []
        self.best_f1_scores = []

        self._setup_directories()
        self._setup_logging()
        self.criterion = NoisyCrossEntropyLoss()

    def predict(self, model, device, loader):
        model.eval()
        y_pred = []
        with torch.no_grad():
            for data in loader:
                data = data.to(device)
                z, mu, logvar, class_logits = model(data.x, data.edge_index, data.edge_attr, data.batch, eps=0.0)
                pred = class_logits.argmax(dim=1)
                y_pred.extend(pred.tolist())
        return y_pred

    def _setup_directories(self):
        self.output_dir = '/kaggle/working/output'
        os.makedirs(self.output_dir, exist_ok=True)
        for d in ['file_checkpoints', 'best', 'file_log']:
            os.makedirs(os.path.join(self.output_dir, d), exist_ok=True)


    def _setup_logging(self):
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        log_filename = os.path.join(self.output_dir, 'file_log', f'training_{self.config.folder_name}_{timestamp}.log')
    
        for handler in logging.root.handlers[:]:
            logging.root.removeHandler(handler)
    
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s [%(levelname)s] %(message)s',
            handlers=[
                logging.FileHandler(log_filename, mode='w'),
                logging.StreamHandler()
            ]
        )
        
        logging.info("Logger created.")

    def load_pretrained(self):
        if self.config.pretrain_paths is not None:
            path = self.config.pretrain_paths
        if path.endswith('.pth'):
            self.pretrain_models = [path]
        else:
            with open(path, 'r') as f:
                self.pretrain_models = [line.strip() for line in f if line.strip()]

    def evaluate_model(self, model: torch.nn.Module, data_loader: DataLoader) -> Dict[str, float]:
        model.eval()
        total_loss, total_samples = 0.0, 0
        all_preds, all_labels = [], []
    
        with torch.no_grad():
            for data in data_loader:
                data = data.to(self.device)
                _, _, _, logits = model(data.x, data.edge_index, data.edge_attr, data.batch)
                loss = self.criterion(logits, data.y)
                preds = logits.argmax(dim=1).cpu().numpy()
                labels = data.y.cpu().numpy()
    
                all_preds.extend(preds)
                all_labels.extend(labels)
    
                batch_size = data.y.size(0)
                total_loss += loss.item() * batch_size
                total_samples += batch_size
    
        avg_loss = total_loss / total_samples
        f1 = f1_score(all_labels, all_preds, average='weighted')
        acc = accuracy_score(all_labels, all_preds)
        return {
            'cross_entropy_loss': avg_loss,
            'f1_score': f1,
            'accuracy': acc,
            'num_samples': total_samples
        }




    def train_single_cycle(self, cycle_num: int, train_data, val_data):


        model = EdgeVGAE(1, 7, self.config.hidden_dim,
                         self.config.latent_dim,
                         self.config.num_classes).to(self.device)

         # Load pretrained models if any
        if len(self.pretrain_models)>0:
            n = len(self.pretrain_models)
            model_file = self.pretrain_models[(cycle_num-1)%n]
            model_data = torch.load(model_file, weights_only=False,map_location=torch.device(self.device))
            model.load_state_dict(model_data['model_state_dict'])
            logging.info(f"Loaded pretrained model: {model_file}")


        train_loader = DataLoader(train_data, batch_size=self.config.batch_size, shuffle=True)
        val_loader = DataLoader(val_data, batch_size=self.config.batch_size, shuffle=False)

        optimizer = torch.optim.Adam(model.parameters(), lr=self.config.learning_rate)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode='max',   
            factor=0.6,  
            patience=7,  
            min_lr=1e-6
        )
        warmup_epochs = self.config.warmup
        best_val_loss, best_f1, epoch_best = float('inf'), 0.0, 0
        best_model_path = None

        for epoch in range(self.config.epochs):
            if epoch < warmup_epochs:
                warm_up_lr(epoch, warmup_epochs, self.config.learning_rate, optimizer)
       

            train_loss, train_acc = self.train_epoch(model, train_loader, optimizer)
            val_metrics = self.evaluate_model(model, val_loader)
            val_loss = val_metrics['cross_entropy_loss']
            val_acc = val_metrics['accuracy']
            val_f1 = val_metrics['f1_score']
            
            if (epoch + 1) % 10 == 0:
                logging.info(
                    f"[Epoch {epoch + 1}] "
                    f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
                    f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}"
                )
        
            # Checkpoint ogni 5 epoche
            if (epoch + 1) % 5 == 0:
                ckpt_path = os.path.join(self.output_dir, 'file_checkpoints', f'ckpt_cycle_{cycle_num}_epoch_{epoch + 1}.pth')

                torch.save({
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'epoch': epoch,
                    'val_loss': val_loss,
                    'val_f1': val_f1,
                    'train_loss': train_loss,
                    'config': self.config
                }, ckpt_path)
      
            
            print(f"Epoch {epoch + 1}: Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
                  f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}")


            if epoch >= warmup_epochs:
                scheduler.step(val_metrics['f1_score'])

            if val_metrics['f1_score'] > best_f1:
                best_val_loss = val_metrics['cross_entropy_loss']
                best_f1 = val_metrics['f1_score']
                epoch_best = epoch

                best_model_path = os.path.join(
                    self.output_dir, 'best',
                    f"best_model_{self.config.folder_name}_cycle_{cycle_num}.pth"
                )


                torch.save({
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'epoch': epoch,
                    'val_loss': best_val_loss,
                    'val_f1': best_f1,
                    'train_loss': train_loss,
                    'config': self.config
                }, best_model_path)


            if (epoch - epoch_best) > self.config.early_stopping_patience // 2 and epoch % 10 == 0:
                checkpoint = torch.load(best_model_path, weights_only=False, map_location=self.device)
                model.load_state_dict(checkpoint['model_state_dict'])

            if (epoch - epoch_best) > self.config.early_stopping_patience:
                break

        self.models.append(best_model_path)
        return best_val_loss, best_f1, best_model_path

    def train_epoch(self, model, train_loader, optimizer):
        model.train()
        total_loss, total_samples = 0.0, 0
        correct = 0
    
        for data in tqdm(train_loader, desc="Training", leave=False):
            data = data.to(self.device)
            optimizer.zero_grad()
    
            z, mu, logvar, logits = model(data.x, data.edge_index, data.edge_attr, data.batch)
            recon_loss = model.recon_loss(z, data.edge_index, data.edge_attr)
            kl_loss = model.kl_loss(mu, logvar)
            class_loss = self.criterion(logits, data.y)
    
            loss = 0.15 * recon_loss + 0.1 * kl_loss + class_loss
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
    
            preds = logits.argmax(dim=1)
            correct += (preds == data.y).sum().item()
    
            batch_size = data.y.size(0)
            total_loss += loss.item() * batch_size
            total_samples += batch_size
    
        avg_loss = total_loss / total_samples
        acc = correct / total_samples
        return avg_loss, acc


    def train_multiple_cycles(self, df, num_cycles=10):
        self.load_pretrained()
        results = []

        for cycle in range(num_cycles):
            cycle_seed = cycle + 1
            set_seed(cycle_seed)

            train_data, val_data = self.prepare_data_split(df, seed=cycle_seed)
            val_loss, val_f1, model_path = self.train_single_cycle(cycle + 1, train_data, val_data)

            results.append({'cycle': cycle + 1, 'seed': cycle_seed,
                            'val_loss': val_loss, 'val_f1': val_f1, 'model_path': model_path})


        model_paths_file = os.path.join(self.output_dir, f"model_paths_{self.config.folder_name}.txt")
        with open(model_paths_file, 'w') as f:

            f.writelines(f"{p}\n" for p in self.models)

        return results

    def get_model_loss(self, model_path: str) -> float:
        checkpoint = torch.load(model_path, weights_only=False, map_location=self.device)
        return checkpoint['val_loss']

    def prepare_data_split(self, df, seed=1):
        db_lst = df.db.unique()
        if len(db_lst) > 1:
            df_train, df_valid = pd.DataFrame(), pd.DataFrame()
            for db in db_lst:
                idx = (df.db == db)
                tmp_train, tmp_valid = train_test_split(df.loc[idx, :], test_size=0.2, shuffle=True, random_state=seed)
                df_train = pd.concat([df_train, tmp_train], ignore_index=True)
                df_valid = pd.concat([df_valid, tmp_valid], ignore_index=True) 
        else:
            df_train, df_valid = train_test_split(df, test_size=0.2, shuffle=True, random_state=seed)

        return create_dataset_from_dataframe(df_train), create_dataset_from_dataframe(df_valid)

    def _compute_ensemble_weights(self, values: np.ndarray, use_loss=True) -> np.ndarray:
        if use_loss:
            weights = np.exp(-values)
        else:
            weights = np.exp(values)
        return weights / np.sum(weights)

    def _ensemble_predict(self, test_df, weight_by='loss'):
        test_dataset = create_dataset_from_dataframe(test_df, result=False)
        test_loader = DataLoader(test_dataset, batch_size=24, shuffle=False)

        all_preds, all_values = [], []

        for model_path in self.models:
            model = EdgeVGAE(1, 7, self.config.hidden_dim,
                            self.config.latent_dim,
                            self.config.num_classes).to(self.device)
            checkpoint = torch.load(model_path, weights_only=False, map_location=self.device)
            model.load_state_dict(checkpoint['model_state_dict'])

            value = checkpoint['val_loss'] if weight_by == 'loss' else checkpoint['val_f1']
            preds = self.predict(model, self.device, test_loader) 

            all_preds.append(preds)
            all_values.append(value)

        all_preds = np.array(all_preds)
        all_values = np.array(all_values)
        weights = self._compute_ensemble_weights(all_values, use_loss=(weight_by == 'loss'))


        num_samples = all_preds.shape[1]
        num_classes = self.config.num_classes
        weighted_votes = np.zeros((num_samples, num_classes))

        for i, preds in enumerate(all_preds):
            for idx, pred_class in enumerate(preds):
                weighted_votes[idx, pred_class] += weights[i]

        ensemble_preds = np.argmax(weighted_votes, axis=1)
        confidence_scores = np.max(weighted_votes, axis=1)

        unique, counts = np.unique(ensemble_preds, return_counts=True)


        return ensemble_preds, confidence_scores


    def predict_with_ensemble(self, test_df):
        return self._ensemble_predict(test_df, weight_by='loss')

    def predict_with_ensemble_score(self, test_df):
        return self._ensemble_predict(test_df, weight_by='score')

    def predict_with_threshold(self, test_df, confidence_threshold=0.5):
        preds, confidences = self.predict_with_ensemble(test_df)
        filtered_preds = np.where(confidences > confidence_threshold, preds, -1)
        return filtered_preds, confidences


In [ ]:
import argparse

def get_user_input(prompt, default=None, required=False, type_cast=str):
    while True:
        user_input = input(f"{prompt} [{default}]: ")

        if user_input == "" and required:
            print("This field is required. Please enter a value.")
            continue

        if user_input == "" and default is not None:
            return default

        if user_input == "" and not required:
            return None

        try:
            return type_cast(user_input)
        except ValueError:
            print(f"Invalid input. Please enter a valid {type_cast.__name__}.")


def get_arguments():
    args = {}
    args['train_path'] = get_user_input("Path to the training dataset (optional)")
    args['test_path'] = get_user_input("Path to the test dataset", required=True)
    args['num_cycles'] = get_user_input("Number cycles [default:5]", required=False, default = 5)
    return argparse.Namespace(**args)


def populate_args(args):
    print("\nArguments received:")
    for key, value in vars(args).items():
        print(f"{key}: {value}")


# Esegui
if __name__ == "__main__":
    args = get_arguments()
    populate_args(args)

In [ ]:
# main.py
import pandas as pd
import torch
import os

def main():
    args = get_arguments()

    if args.train_path:
        folder = os.path.basename(os.path.dirname(args.train_path))
    elif args.test_path:
        folder = os.path.basename(os.path.dirname(args.test_path))
    else:
        raise ValueError("You must provide at least --train_path or --test_path.")

    if args.train_path is not None:
        
        default_pretrain = (
            f"/kaggle/working/GNN_classification_noisy_label/"
            f"model_for_train/model_{folder}.pth"
        )
       
        if not os.path.exists(default_pretrain):
            raise FileNotFoundError(
                f"Expected pretrained checkpoint at '{default_pretrain}', but not found."
            )
            
        config = ModelConfig(
            test_path=args.test_path,
            train_path=args.train_path,
            num_cycles=args.num_cycles,
            pretrain_paths=default_pretrain,   
        )
        
    else:
        
        config = ModelConfig(
            test_path=args.test_path,
            train_path=None,
            num_cycles=args.num_cycles,
            pretrain_paths=None,
        )

    device = "cuda" if torch.cuda.is_available() else "cpu"
    trainer = ModelTrainer(config, device)

    if args.train_path:
        print(f"Entering training mode on folder '{folder}' …")
        df_train = load_dataset(args.train_path)
        trainer.train_multiple_cycles(df_train, args.num_cycles)

        if args.test_path:
            
            print("Training complete, now running inference on test set …")
            df_test = load_dataset(args.test_path)
            predictions, _ = trainer.predict_with_ensemble_score(df_test)

            # Save predictions to a simple ./output folder
            output_dir = "./output"
            os.makedirs(output_dir, exist_ok=True)
            out_csv = os.path.join(output_dir, f"testset_{folder}.csv")
            pd.DataFrame({"id": range(len(predictions)), "pred": predictions}).to_csv(
                out_csv, index=False
            )
            print(f"Predictions saved to: {out_csv}")
    else:
        print(f"Entering inference mode for folder '{folder}' …")

        model_paths = []
        for cycle in range(1, 6):  
            path = f"/kaggle/working/GNN_classification_noisy_label/best_models/{folder}/best_model_{folder}_cycle_{cycle}.pth"
            if not os.path.exists(path):
                raise FileNotFoundError(f"Expected checkpoint at '{path}', but not found.")
            model_paths.append(path)
        
        trainer.models = model_paths

        if not args.test_path:
            raise ValueError("Inference mode requires --test_path to be provided.")

        df_test = load_dataset(args.test_path)
        print("Generating predictions on test set …")
        predictions, _ = trainer.predict_with_ensemble_score(df_test)

        output_dir = "./predictions"
        os.makedirs(output_dir, exist_ok=True)
        out_csv = os.path.join(output_dir, f"testset_{folder}.csv")
        pd.DataFrame({"id": range(len(predictions)), "pred": predictions}).to_csv(
            out_csv, index=False
        )
        print(f"Predictions saved to: {out_csv}")

if __name__ == "__main__":
    main()